In [2]:
import os 
import sys

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-GPU-bb1ccb6e-2bc9-c7a1-b25d-3eef9033e192/3/0"

In [2]:
# import numpy as np
# data = np.load("samples-c100.npz")
# import matplotlib.pyplot as plt
# print(list(data.keys()))

# x = data["x"]

# print(x.shape)
# fix, axs = plt.subplots(5,10, figsize=(10,5))

# axs = axs.flatten()
# for i in range(len(axs)):
#     axs[i].imshow(np.moveaxis(x[i], 0, -1))
#     axs[i].tick_params(left = False, right = False , labelleft = False , labelbottom = False, bottom = False)

# plt.tight_layout()
# plt.savefig("c100-bigan-samples.png", dpi=300)
# plt.show()

# import torch 
# x = torch.tensor(x)

# torch.save(x, "samples-c100.pt")

In [3]:
import torch 
import numpy as np
import matplotlib.pyplot as plt

# x = torch.load("samples-c100.pt")

In [89]:
from pytorch_ood.model import WideResNet
from pytorch_ood.detector import EnergyBased, MaxSoftmax
from pytorch_ood.utils import OODMetrics, ToUnknown, ToRGB
from pytorch_ood.dataset.img import Textures, LSUNCrop, LSUNResize, TinyImageNetCrop, TinyImageNetResize
from torchvision.datasets import CIFAR100
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import Compose, ToPILImage, Normalize, ToTensor, Resize, ToPILImage
import torch 

def test(model):
    
    prep = Compose([
        ToRGB(),
        Resize([32,32]),
        ToTensor(),
        Normalize(
            std=[0.24705882352941176470, 0.24352941176470588235, 0.26156862745098039215], 
            mean=[0.49137254901960784313, 0.48235294117647058823, 0.44666666666666666666]), 
        ])

    metrics = [] 
    
    with torch.no_grad():
        model.eval()

        data_in = CIFAR100(root="/data_fast/kirchheim/datasets/", download=True, transform=prep, train=False)

        for Data_clz in [Textures, LSUNCrop, LSUNResize, TinyImageNetCrop, TinyImageNetResize]:
            ood_name = Data_clz.__name__

            data_out = Data_clz(root="/data_fast/kirchheim/datasets/", transform=prep, download=True, 
                                target_transform=ToUnknown())

            data_loader = DataLoader(data_in + data_out, batch_size=128, num_workers=10)

            # create detector
            softmax = MaxSoftmax(model)
            energy = EnergyBased(model)

            # evaluate
            metric_softmax = OODMetrics()
            metric_energy = OODMetrics()

            for x, y in data_loader:
                logits = model(x.cuda())
                metric_softmax.update(softmax.score(logits), y)
                metric_energy.update(energy.score(logits), y)

            # print(ood_name, metric_softmax.compute())

            m = metric_softmax.compute()
            m.update({
                "Dataset": ood_name,
                "Method": "Softmax"
            })
            metrics.append(m)
            
            m = metric_energy.compute()
            m.update({
                "Dataset": ood_name,
                "Method": "Energy"
            })
            metrics.append(m)
    
    model.train()
    
    return metrics
        

In [90]:
# create Neural Network
model = WideResNet(pretrained="cifar100-pt", num_classes=100).eval().cuda()

metrics = test(model)

Files already downloaded and verified


/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: `pos_label` automatically set 1.
  warnings.warn(*args, **kwargs)
/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/functional/classification/precision_recall_curve.py:156: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().

In [97]:
data = pd.DataFrame(metrics)
data
print((data.groupby(["Method"]).mean() * 100).to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrr}
\toprule
{} &  AUROC &  AUPR-IN &  AUPR-OUT &  ACC95TPR &  FPR95TPR \\
Method  &        &          &           &           &           \\
\midrule
Energy  &  84.41 &    79.72 &     86.85 &     71.38 &     48.55 \\
Softmax &  78.71 &    73.35 &     82.58 &     66.36 &     58.58 \\
\bottomrule
\end{tabular}



In [99]:
from pytorch_ood.model import WideResNet
from pytorch_ood.detector import EnergyBased, MaxSoftmax
from pytorch_ood.utils import OODMetrics
from pytorch_ood.dataset.img import Textures
from torchvision.datasets import CIFAR10 
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import Compose, ToPILImage, Normalize, ToTensor, Resize, ToPILImage
import torch 

from pytorch_ood.loss import OutlierExposureLoss
from torch.optim import SGD
from tqdm import tqdm

torch.manual_seed(10)

mt = Compose([
    ToPILImage(),
    Resize([32,32]),
    ToTensor(),
    Normalize(
        std=[0.24705882352941176470, 0.24352941176470588235, 0.26156862745098039215], 
        mean=[0.49137254901960784313, 0.48235294117647058823, 0.44666666666666666666]), 
    ])

mt2 = Compose([
    Resize([32,32]),
    ToTensor(),
    Normalize(
        std=[0.24705882352941176470, 0.24352941176470588235, 0.26156862745098039215], 
        mean=[0.49137254901960784313, 0.48235294117647058823, 0.44666666666666666666]), 
    ])

# prepare data 
# data_in_test = CIFAR100(root=".", download=True, transform=mt2, train=False)
# data_out_test = Textures(root=".", transform=mt2, download=True)
# test_loader = DataLoader(data_in_test + data_out_test, batch_size=512, num_workers=20)

# TODO: use 80,000?
x = torch.load("samples-c100.pt")[:80000]
gan_oe_data = torch.stack([mt(t) for t in x])
gan_oe_label = torch.ones(size=(len(x),)).long() * -1

data_in_train = CIFAR100(root=".", download=True, transform=mt2, target_transform=torch.tensor)
data_out_train = TensorDataset(gan_oe_data,gan_oe_label)
train_loader = DataLoader(data_in_train + data_out_train, batch_size=256, shuffle=True, num_workers=20)
      
# create Neural Network
model = WideResNet(pretrained="cifar100-pt", num_classes=100).cuda()
opti = SGD(model.parameters(), lr=0.001)

criterion = OutlierExposureLoss(alpha=0.5)

mav = 0.0
mavs = []

results = []

for i in range(5):
    print(f"Epoch {i}")
    
    # train 
    model.train()
    bar = tqdm(train_loader)
    for x, y in bar:
        opti.zero_grad()
        y_hat = model(x.cuda())
        loss = criterion(y_hat, y.cuda())
        mav = 0.2 * loss.item() + 0.8 * mav
        mavs.append(mav)

        loss.backward()
        opti.step()
        bar.set_postfix({"loss": mav})
        
    print(80*"#")
    
    r = test(model)
    for entry in r:
        entry.update({"Epoch": i})
    
    metrics_goe += r


Files already downloaded and verified
Epoch 0


100%|████████████████████████████████████████████████████████| 391/391 [02:01<00:00,  3.23it/s, loss=1.95]


Files already downloaded and verified
################################################################################
Epoch 1


100%|████████████████████████████████████████████████████████| 391/391 [02:01<00:00,  3.23it/s, loss=1.74]


Files already downloaded and verified
################################################################################
Epoch 2


...

Files already downloaded and verified
################################################################################
Epoch 3


100%|████████████████████████████████████████████████████████| 391/391 [02:01<00:00,  3.23it/s, loss=1.52]


Files already downloaded and verified
################################################################################
Epoch 4


100%|████████████████████████████████████████████████████████| 391/391 [02:01<00:00,  3.23it/s, loss=1.48]


Files already downloaded and verified
################################################################################


In [109]:
data_goe = pd.DataFrame(results[4])
data_goe
print((data_goe.groupby(["Method"]).mean() * 100).to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrrr}
\toprule
{} &  AUROC &  AUPR-IN &  AUPR-OUT &  ACC95TPR &  FPR95TPR &  Epoch \\
Method  &        &          &           &           &           &        \\
\midrule
Energy  &  90.73 &    86.43 &     93.01 &     80.69 &     30.43 & 400.00 \\
Softmax &  89.08 &    85.30 &     91.39 &     76.98 &     38.02 & 400.00 \\
\bottomrule
\end{tabular}



In [114]:
"""
Do it with uniformely distributed outlier samples 
"""
from pytorch_ood.model import WideResNet
from pytorch_ood.detector import EnergyBased, MaxSoftmax
from pytorch_ood.utils import OODMetrics
from pytorch_ood.dataset.img import Textures
from torchvision.datasets import CIFAR10 
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import Compose, ToPILImage, Normalize, ToTensor, Resize, ToPILImage
import torch 

from pytorch_ood.loss import OutlierExposureLoss
from torch.optim import SGD
from tqdm import tqdm

torch.manual_seed(10)

# TODO: use 80,000?
x = torch.randint(0, 255, size=(80000, 3, 32, 32), dtype=torch.uint8) #  torch.load("samples.pt")[:80000]
gan_oe_data = torch.stack([mt(t) for t in x])
gan_oe_label = torch.ones(size=(len(x),)).long() * -1

data_in_train = CIFAR100(root=".", download=True, transform=mt2, target_transform=torch.tensor)
data_out_train = TensorDataset(gan_oe_data,gan_oe_label)
train_loader = DataLoader(data_in_train + data_out_train, batch_size=256, shuffle=True, num_workers=20)
      
# create Neural Network
model = WideResNet(pretrained="cifar100-pt", num_classes=100).cuda()
opti = SGD(model.parameters(), lr=0.001)

# print(y.shape)
# print(x.shape)

criterion = OutlierExposureLoss(alpha=0.5)

mav = 0.0
mavs = []

results_noise = []

for i in range(5):
    print(f"Epoch {i}")
    # train 
    model.train()
    bar = tqdm(train_loader)
    for x, y in bar:
        opti.zero_grad()
        y_hat = model(x.cuda())
        loss = criterion(y_hat, y.cuda())
        mav = 0.2 * loss.item() + 0.8 * mav
        mavs.append(mav)

        loss.backward()
        opti.step()
        bar.set_postfix({"loss": mav})
    
    r = test(model)
    for entry in r:
        entry.update({"Epoch": i})
        
    results_noise += r
       
    print(80*"#")


Files already downloaded and verified
Epoch 0


100%|████████████████████████████████████████████████████████| 508/508 [02:36<00:00,  3.25it/s, loss=1.77]


Files already downloaded and verified


/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: `pos_label` automatically set 1.
  warnings.warn(*args, **kwargs)
/home/kirchheim/anaconda3/envs/myenv/lib/python3.8/site-packages/torchmetrics/functional/classification/precision_recall_curve.py:156: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().

################################################################################
Epoch 1


...

Files already downloaded and verified
################################################################################
Epoch 3


100%|████████████████████████████████████████████████████████| 508/508 [02:36<00:00,  3.24it/s, loss=1.55]


Files already downloaded and verified
################################################################################
Epoch 4


100%|████████████████████████████████████████████████████████| 508/508 [02:36<00:00,  3.25it/s, loss=1.55]


Files already downloaded and verified
################################################################################


In [117]:
df_noise = pd.DataFrame(results_noise)
tmp = df_noise[df_noise["Epoch"] == 4]
print((tmp.groupby(["Method"]).mean() * 100).to_latex(float_format="%.2f"))

\begin{tabular}{lrrrrrr}
\toprule
{} &  AUROC &  AUPR-IN &  AUPR-OUT &  ACC95TPR &  FPR95TPR &  Epoch \\
Method  &        &          &           &           &           &        \\
\midrule
Energy  &  91.59 &    89.03 &     93.28 &     79.56 &     33.04 & 400.00 \\
Softmax &  81.56 &    77.33 &     85.27 &     69.03 &     53.56 & 400.00 \\
\bottomrule
\end{tabular}



In [ ]:
"""
Do it with the usual (non-synthetic outliers)
"""
from pytorch_ood.model import WideResNet
from pytorch_ood.detector import EnergyBased, MaxSoftmax
from pytorch_ood.utils import OODMetrics
from pytorch_ood.dataset.img import Textures, TinyImages300k
from torchvision.datasets import CIFAR10 
from torch.utils.data import TensorDataset, DataLoader
from torchvision.transforms import Compose, ToPILImage, Normalize, ToTensor, Resize, ToPILImage
import torch 

from pytorch_ood.loss import OutlierExposureLoss
from torch.optim import SGD
from tqdm import tqdm

torch.manual_seed(10)

data_out_train = TinyImages300k(root="/data_fast/kirchheim/datasets/", transform=mt2, download=True, 
                                target_transform=ToUnknown())

data_in_train = CIFAR100(root=".", download=True, transform=mt2)
train_loader = DataLoader(data_in_train + data_out_train, batch_size=256, shuffle=True, num_workers=20)
      
# create Neural Network
model = WideResNet(pretrained="cifar100-pt", num_classes=100).cuda()
opti = SGD(model.parameters(), lr=0.001)

criterion = OutlierExposureLoss(alpha=0.5)

mav = 0.0
mavs = []

results_real_outliers = []

for i in range(10):
    print(f"Epoch {i}")
    # train 
    model.train()
    bar = tqdm(train_loader)
    for x, y in bar:
        opti.zero_grad()
        y_hat = model(x.cuda())
        loss = criterion(y_hat, y.cuda())
        mav = 0.2 * loss.item() + 0.8 * mav
        mavs.append(mav)

        loss.backward()
        opti.step()
        bar.set_postfix({"loss": mav})
    
    r = test(model)
    for entry in r:
        entry.update({"Epoch": i})
        
    results_real_outliers += r
       
    print(80*"#")

Files already downloaded and verified
Epoch 0


 41%|██████████████████████▎                                | 556/1368 [02:51<04:04,  3.31it/s, loss=3.28]

In [ ]:
df_real = pd.DataFrame(results_real_outliers)
tmp = df_real[df_real["Epoch"] == 4]
print((tmp.groupby(["Method"]).mean() * 100).to_latex(float_format="%.2f"))

In [ ]:
sb.lineplot(data=df_real.groupby(["Method", "Epoch"]).mean(), x="Epoch", y="AUROC", hue="Method")

In [74]:
df_real_outliers

,Metric,Value,Epoch,Method
0,AUROC,0.787129,0,MaxSoftmax
1,AUROC,0.844067,0,Energy
2,AUROC,0.721468,1,MaxSoftmax
3,AUROC,0.664469,1,Energy
4,AUROC,0.768718,2,MaxSoftmax
5,AUROC,0.718181,2,Energy
6,AUROC,0.810844,3,MaxSoftmax
7,AUROC,0.774762,3,Energy
8,AUROC,0.829380,4,MaxSoftmax
9,AUROC,0.799271,4,Energy


In [76]:
total_df = pd.concat([df, df_real_outliers, df_uniform])

In [77]:
sb.lineplot(data  = total_df, 

,Metric,Value,Epoch,Method
0,AUROC,0.787129,0,MaxSoftmax
1,AUROC,0.844067,0,Energy
2,AUROC,0.874104,1,MaxSoftmax
3,AUROC,0.914078,1,Energy
4,AUROC,0.885754,2,MaxSoftmax
...,...,...,...,...
5,AUROC,0.881609,2,Energy
6,AUROC,0.806031,3,MaxSoftmax
7,AUROC,0.901618,3,Energy
8,AUROC,0.807461,4,MaxSoftmax


In [78]:
df

,Metric,Value,Epoch,Method
0,AUROC,0.787129,0,MaxSoftmax
1,AUROC,0.844067,0,Energy
2,AUROC,0.874104,1,MaxSoftmax
3,AUROC,0.914078,1,Energy
4,AUROC,0.885754,2,MaxSoftmax
5,AUROC,0.916879,2,Energy
6,AUROC,0.888863,3,MaxSoftmax
7,AUROC,0.913140,3,Energy
8,AUROC,0.890022,4,MaxSoftmax
9,AUROC,0.910107,4,Energy
